In [65]:
#!pip install pandas
from openpyxl import load_workbook
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
#import csv

In [66]:
#파일이 연도 별로 나뉘어져 있다. ex weather2020, weather2019, ...
#날씨를 딕셔너리에 기록
def weatherFileToDict(filename, dict_weather):
    book = load_workbook(filename)
    sheet = book.active
    rows = list(sheet.rows)[1:]
    
    #rows : 모든 샘플, row : 샘플 하나
    for row in rows:
        # ex) year = '2020'  month = '1월'  day = '1일'
        ymd = row[0].value
        year = ymd[:4]
        month = ymd[5:7]+'월'
        if month[0] == '0':
            month = month[1:]
        day = ymd[8:10]+'일'
        if day[0] == '0':
            day = day[1:]
        
        # 지역이름
        area = row[1].value
        
        #딕셔너리 구조 만들기 key: year->area->month->day = []
        if dict_weather.get(year) == None:
            dict_weather[year] = {}
        if dict_weather[year].get(area) == None:
            dict_weather[year][area] = {}
        if dict_weather[year][area].get(month) == None:
            dict_weather[year][area][month] = {}
        if dict_weather[year][area][month].get(day) == None:
            dict_weather[year][area][month][day] = []
        
        # 각 날짜 별로 평균기온 ~~ 습도, 일조량, 운량 전부 리스트에 append
        for col in row[2:]:
            dict_weather[year][area][month][day].append(col.value)




In [67]:
# 해당 팀의 해당 날짜 및 시간에 출전한 타자의 순서와 이름을 기록한다.
# 해당 팀의 타순을 리스트로 반환
def getBatter(year, month, day, time, team):
    #ex) month = '01', day = '01'
    if len(month) == 1:
        month = '0'+month
    if len(day) == 1:
        day = '0'+day
    ymd = year + '-' + month + '-' + day
    #http://www.statiz.co.kr/team.php?opt=0&sopt=4&year=2018&team=KIA&mdate=2018-08-08
    url = 'http://www.statiz.co.kr/team.php?opt=0&sopt=4&year='+year+'&team='+team+'&mdate='+ymd
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 해당 연월일에 맞는 타순 데이터 담긴 박스에 접근
    data_box = soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div')
    box = data_box[0]
    print(ymd +', '+ team + ' : ' + str(data_box[0].select_one('h3')))
    #만약 같은 연월일은 맞으나 시간이 다르다면 맞는 시간에 대한 박스에 접근
    if data_box[0].select_one('h3 > font').text != time:
        box = data_box[1]
    #해당 박스의 <a>들 모두 저장 
    atags = box.select('table > tbody > tr > td > a')
    
    # <a>로부터 타자의 이름과 생년월일 기록 (나중에 해당 선수의 정보를 가지고 있는 웹페이지의 url만들 때 사용)
    # cnt는 타자 9명만을 기록하기 위해 사용
    cnt = 0
    list_batter = []
    for atag in atags:
        name = atag.text
        index_birth = atag['href'].find('birth=')+len('birth=')
        name+='&'+atag['href'][ index_birth : index_birth+len('0000-00-00')]
        list_batter.append(name)
        cnt+=1
        if cnt == 9:
            break

    driver.quit()
    return list_batter
    

In [68]:
# 경기 마다의 기본 정보를 딕셔너리로 기록
def matchFileToDict(filename, dict_match, dict_homeAway):
    file = pd.read_csv(filename)

    for value_list in file.values:
        # 요일 제거 ex) 10.19(월) -> ['10', '19'] 
        if type(value_list[2]) == type(''):
            date = value_list[2].split('(')[0].split('.')
        
        # 취소된 경기는 기록하지 않는다. 정규시즌만 기록한다.
        if value_list[10] == '우천취소' or value_list[10] == '미세먼지취소' or value_list[-1] != '0,9':
            continue
        
        # ex) year = '2020'
        year = repr(value_list[1])
        
        #나중에 지울 부분
        if year != '2019':
            continue
        
        # ex) date = ['08', '01']
        # month = '8월'  day = '1일'
        month = date[0]+'월'
        if month[0] == '0':
            month = month[1:]
        day = date[1]+'일'
        if day[0] == '0':
            day = day[1:]
        
        #시간이 없는 경우가 하나 있는데, 이경우 찾아보니 15:00였음
        time = value_list[3]
        if type(time) != type(''):
            time = '15:00'
        field = value_list[9]
        
        #두산6vs7넥센 -> ['두산6', '7넥센']
        str_list = value_list[4].split('v')
        str_list[1] = str_list[1][1:]
        if not str_list[0][-1].isdigit(): #점수가 없는 경기는 승패를 가릴 수 없어서 넘어가기
             continue
#        match = str_list[0] + ' ' + str_list[1]
        
        #['두산6', '7넥센'] -> match = '두산 넥센', num = ['6','7']
        match = ''
        num = []
        for s in str_list:
            num_string = ''
            for ch in s:
                if ch.isdigit():
                    num_string += ch
                    continue
                match+=ch
            num.append(num_string)
            match+=' '
              
        if num[0] < num[1]:
            temp = num[0]
            num[0] = num[1]
            num[1] = temp
            match = match.split(' ')[1] + ' ' + match.split(' ')[0]
        
        # 올스타전팀의 이름은 나눔, 드림인데 이 정보는 넘어간다.
        if match.split(' ')[0] == '나눔' or match.split(' ')[0] == '드림' or match.split(' ')[1] == '나눔' or match.split(' ')[1]=='드림':
            continue
        
        homeAway = [0, 0] # 0: away, 1: home
        #이긴팀 홈?어웨이?
        for f in dict_homeAway[match.split(' ')[0]]:
            if f == field:
                homeAway[0] = 1
        #진팀 홈?어웨이?
        for f in dict_homeAway[match.split(' ')[1]]:
            if f == field:
                homeAway[1] = 1
            
        
        #기록전 딕셔너리 구조 만들기 key : '2020' -> '1월' -> '1일' -> '구장이름' -> '두산 넥센' -> '시간' = []
        if dict_match.get(year)==None:
            dict_match[year] = {}
        if dict_match[year].get(month)==None:
            dict_match[year][month] = {}
        if dict_match[year][month].get(day)==None:
            dict_match[year][month][day] = {}
        if dict_match[year][month][day].get(field) == None:
            dict_match[year][month][day][field] = {}
        if dict_match[year][month][day][field].get(match) == None:
            dict_match[year][month][day][field][match] = {}
        if dict_match[year][month][day][field][match].get(time) == None:
            dict_match[year][month][day][field][match][time] = []
        
        #각각 이긴 팀, 이긴 팀 점수, 진 팀, 진 팀 점수 append
        dict_match[year][month][day][field][match][time].append(match.split(' ')[0])
        dict_match[year][month][day][field][match][time].append(num[0])
        dict_match[year][month][day][field][match][time].append(homeAway[0])
        dict_match[year][month][day][field][match][time].append(match.split(' ')[1])
        dict_match[year][month][day][field][match][time].append(num[1])
        dict_match[year][month][day][field][match][time].append(homeAway[1])
        
        # 해당 날짜, 시간, 팀에 대한 타순 기록 (선수는 '이름&생년월일'로 기록한다. 추 후 선수기록 웹페이지의 url만들 때 쓰인다.)
        list_temp = getBatter(year, month[:-1], day[:-1], time, match.split(' ')[0])
        for elem in list_temp:
            dict_match[year][month][day][field][match][time].append(elem)
        list_temp = getBatter(year, month[:-1], day[:-1], time, match.split(' ')[1])
        for elem in list_temp:
            dict_match[year][month][day][field][match][time].append(elem)

In [69]:
#날씨와 경기에 대한 딕셔너리를 합병한다. (to dict_match)
def mergeDicts(dict_match, dict_weather, dict_area):
    for year in dict_match.keys():
        #나중에 지울부분
        if year != '2019':
            continue
        for month in dict_match[year].keys():
            for day in dict_match[year][month].keys():
                for field in dict_match[year][month][day]:
                    for match in dict_match[year][month][day][field]:
                        for time in dict_match[year][month][day][field][match]:
                            if field == '고척':
                                for i in range(4):# 날씨(기온 등) 정보만 만 '-'로
                                    dict_match[year][month][day][field][match][time].append('-')
                                continue
                            area = dict_area[field]
                            list_weather_all = dict_weather[year][area][month][day]
                            dict_match[year][month][day][field][match][time].append(list_weather_all[0]) #평균기온
                            if list_weather_all[3] == '-':
                                list_weather_all[3] = '0.0'
                            dict_match[year][month][day][field][match][time].append(list_weather_all[3]) #강수량
                            dict_match[year][month][day][field][match][time].append(list_weather_all[4]) #평균풍속
                            dict_match[year][month][day][field][match][time].append(list_weather_all[5]) #상대습도

잠실, 고척, 수원, 사직, 광주, 문학, 창원, 대구, 대전, 울산, 마산, 포항, 청주  
서울, 서울, 수원, 부산, 광주, 인천, 창원, 대구, 대전, 울산, 서울, 포항, 청주  
  
KT | 한화   | LG| SK |키움 | 두산 | NC | KIA| 삼성     |롯데  
수원|대전,청주| 잠실|문학|고척 | 잠실 | 창원 | 광주| 대구,포항 |사직,울산

In [70]:
dict_area={'잠실':'서울', '고척':'서울', '수원':'수원', '사직':'부산', '광주':'광주', '문학':'인천', '창원':'창원', '대구':'대구', '대전':'대전', '울산':'울산', '마산':'창원', '포항':'포항', '청주':'청주'}
dict_homeAway={'KT':['수원'], '한화':['대전','청주'], 'LG':['잠실'], 'SK':['문학'], '키움':['고척'], '두산':['잠실'], 'NC':['창원','마산'], 'KIA':['광주'], '삼성':['대구','포항'], '롯데':['사직','울산'], '넥센':['고척'], '드림':[''], '나눔':['']}

In [71]:
#match_data 파일읽어서 기록
dict_match = {}
filename = './match_data.csv'
matchFileToDict(filename, dict_match, dict_homeAway)
print(dict_match)

2019-03-23, 두산 : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, 한화 : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, SK : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, KT : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, 키움 : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, 롯데 : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, NC : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, 삼성 : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, LG : <h3 class="box-title"><b>2019-03-23 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-23, KIA : <h3 class="box-title"><b>201

2019-03-31, NC : <h3 class="box-title"><b>2019-03-31 </b> <font style="font-size:12px;">14:00</font></h3>
2019-03-31, 한화 : <h3 class="box-title"><b>2019-03-31 </b> <font style="font-size:12px;">14:00</font></h3>
2019-04-02, 두산 : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, KT : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, 롯데 : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, SK : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, KIA : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, 삼성 : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, 키움 : <h3 class="box-title"><b>2019-04-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-02, NC : <h3 class="box-title"><b>201

2019-04-11, KT : <h3 class="box-title"><b>2019-04-11 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-11, 키움 : <h3 class="box-title"><b>2019-04-11 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-11, SK : <h3 class="box-title"><b>2019-04-11 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-11, 한화 : <h3 class="box-title"><b>2019-04-11 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, LG : <h3 class="box-title"><b>2019-04-12 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, 두산 : <h3 class="box-title"><b>2019-04-12 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, KIA : <h3 class="box-title"><b>2019-04-12 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, SK : <h3 class="box-title"><b>2019-04-12 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, 삼성 : <h3 class="box-title"><b>2019-04-12 </b> <font style="font-size:12px;">18:30</font></h3>
2019-04-12, KT : <h3 class="box-title"><b>201

2019-04-20, 롯데 : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-20, KT : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-20, 두산 : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-20, KIA : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-20, 삼성 : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-20, 한화 : <h3 class="box-title"><b>2019-04-20 </b> <font style="font-size:12px;">17:00</font></h3>
2019-04-21, LG : <h3 class="box-title"><b>2019-04-21 </b> <font style="font-size:12px;">14:00</font></h3>
2019-04-21, 키움 : <h3 class="box-title"><b>2019-04-21 </b> <font style="font-size:12px;">14:00</font></h3>
2019-04-21, SK : <h3 class="box-title"><b>2019-04-21 </b> <font style="font-size:12px;">14:00</font></h3>
2019-04-21, NC : <h3 class="box-title"><b>201

2019-05-01, KIA : <h3 class="box-title"><b>2019-05-01 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-01, 삼성 : <h3 class="box-title"><b>2019-05-01 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-01, 한화 : <h3 class="box-title"><b>2019-05-01 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-01, 두산 : <h3 class="box-title"><b>2019-05-01 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, LG : <h3 class="box-title"><b>2019-05-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, KT : <h3 class="box-title"><b>2019-05-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, SK : <h3 class="box-title"><b>2019-05-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, 키움 : <h3 class="box-title"><b>2019-05-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, NC : <h3 class="box-title"><b>2019-05-02 </b> <font style="font-size:12px;">18:30</font></h3>
2019-05-02, 롯데 : <h3 class="box-title"><b>201

...

2019-08-13, 한화 : <h3 class="box-title"><b>2019-08-13 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-13, NC : <h3 class="box-title"><b>2019-08-13 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, 키움 : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, LG : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, KT : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, 롯데 : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, KIA : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, 두산 : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, 한화 : <h3 class="box-title"><b>2019-08-14 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-14, NC : <h3 class="box-title"><b>201

2019-08-23, 롯데 : <h3 class="box-title"><b>2019-08-23 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-23, KT : <h3 class="box-title"><b>2019-08-23 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-23, 키움 : <h3 class="box-title"><b>2019-08-23 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-23, KIA : <h3 class="box-title"><b>2019-08-23 </b> <font style="font-size:12px;">18:30</font></h3>
2019-08-24, KT : <h3 class="box-title"><b>2019-08-24 </b> <font style="font-size:12px;">18:00</font></h3>
2019-08-24, LG : <h3 class="box-title"><b>2019-08-24 </b> <font style="font-size:12px;">18:00</font></h3>
2019-08-24, SK : <h3 class="box-title"><b>2019-08-24 </b> <font style="font-size:12px;">18:00</font></h3>
2019-08-24, KIA : <h3 class="box-title"><b>2019-08-24 </b> <font style="font-size:12px;">18:00</font></h3>
2019-08-24, NC : <h3 class="box-title"><b>2019-08-24 </b> <font style="font-size:12px;">18:00</font></h3>
2019-08-24, 롯데 : <h3 class="box-title"><b>20

2019-09-01, KIA : <h3 class="box-title"><b>2019-09-01 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-01, 롯데 : <h3 class="box-title"><b>2019-09-01 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-01, 한화 : <h3 class="box-title"><b>2019-09-01 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-01, KT : <h3 class="box-title"><b>2019-09-01 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-03, 키움 : <h3 class="box-title"><b>2019-09-03 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-03, 두산 : <h3 class="box-title"><b>2019-09-03 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-03, SK : <h3 class="box-title"><b>2019-09-03 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-03, NC : <h3 class="box-title"><b>2019-09-03 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-03, 삼성 : <h3 class="box-title"><b>2019-09-03 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-03, 롯데 : <h3 class="box-title"><b>201

2019-09-13, LG : <h3 class="box-title"><b>2019-09-13 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-13, 키움 : <h3 class="box-title"><b>2019-09-13 </b> <font style="font-size:12px;">14:00</font></h3>
2019-09-14, LG : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, KIA : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, SK : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, 두산 : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, NC : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, 삼성 : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, KT : <h3 class="box-title"><b>2019-09-14 </b> <font style="font-size:12px;">17:00</font></h3>
2019-09-14, 키움 : <h3 class="box-title"><b>201

2019-09-26, KIA : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, 롯데 : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, 두산 : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, 삼성 : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, 한화 : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, NC : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, LG : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-26, KT : <h3 class="box-title"><b>2019-09-26 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-27, NC : <h3 class="box-title"><b>2019-09-27 </b> <font style="font-size:12px;">18:30</font></h3>
2019-09-27, LG : <h3 class="box-title"><b>201

In [72]:
# 날씨 정보 파일로부터 읽어서 기록
dict_weather = {}
for year in range(3):
    #나중에 지울부분
    if year != 1:
        continue
    year += 2018
    filename = './weather'+repr(year)+'.xlsx'
    weatherFileToDict(filename, dict_weather)
#print(dict_weather)

In [73]:
# 경기일정, 날씨, 선수들 정보 합치기
# dict_match = merge dict_match and dict_weather  (dict_player는 한 번 스크랩한 선수들을 저장해서 중복 스크랩하는 것을 방지)
dict_player={}
mergeDicts(dict_match, dict_weather, dict_area, dict_player)
print(dict_match)

{'2019': {'3월': {'23일': {'잠실': {'두산 한화': {'14:00': ['두산', '5', 1, '한화', '4', 0, '허경민&1990-08-26', '정수빈&1990-10-07', '박건우&1990-09-08', '김재환&1988-09-22', '오재일&1986-10-29', '페르난데스&1988-04-27', '오재원&1985-02-09', '박세혁&1990-01-09', '김재호&1985-03-21', '정근우&1982-10-02', '송광민&1983-06-24', '호잉&1989-05-18', '김태균&1982-05-29', '이성열&1984-07-13', '하주석&1994-02-25', '김민하&1989-02-25', '최재훈&1989-08-27', '정은원&2000-01-17', '3.3', '1.7', '8.6', '46.9']}}, '문학': {'SK KT': {'14:00': ['SK', '7', 1, 'KT', '4', 0, '노수광&1990-08-06', '한동민&1989-08-09', '최정&1987-02-28', '로맥&1985-09-30', '이재원&1988-02-24', '정의윤&1986-07-25', '김강민&1982-09-13', '최항&1994-01-03', '김성현&1987-03-09', '황재균&1987-07-28', '박경수&1984-03-31', '강백호&1999-07-29', '로하스&1990-05-24', '유한준&1981-07-01', '윤석민&1985-09-04', '오승택&1991-11-18', '장성우&1990-01-17', '배정대&1995-06-12', '3.8', '2.6', '16.9', '54.8']}}, '사직': {'키움 롯데 ': {'14:00': ['키움', '7', 0, '롯데', '4', 1, '서건창&1989-08-22', '김하성&1995-10-17', '박병호&1986-07-10', '샌즈&1987-09-28', '장영석&1990-05-14', '임병욱&1995

In [74]:
#합친 정보 파일에 저장
book = Workbook()
sheet = book.active
sheet.title = 'merge2'
sheet.append(['날짜','구장', '시간', '이긴팀', '이긴팀점수', '이긴팀홈어웨이', '진팀', '진팀점수', '진팀홈어웨이', 'w팀1','w팀2','w팀3','w팀4','w팀5','w팀6','w팀7','w팀8','w팀9', 'l팀1','l팀2','l팀3','l팀4','l팀5','l팀6','l팀7','l팀8','l팀9','평균기온', '강수량', '평균풍속', '상대습도'])
for year in dict_match.keys():
    #나중에 지울부분
    if year != '2019':
        continue
    for month in dict_match[year].keys():
        for day in dict_match[year][month].keys():
            if len(month) == 2:
                m = '0' + month[0]
            else:
                m = month[0:2]
            
            if len(day) == 2:
                d = '0' + day[0]
            else:
                d = day[0:2]
            
            date = year + '-' + m + '-' + d
            for field in dict_match[year][month][day].keys():
                for match in dict_match[year][month][day][field]:
                    for time in dict_match[year][month][day][field][match]:
                        list_cur = [date, field, time]
                        for element in dict_match[year][month][day][field][match][time]:
                            list_cur.append(element)
                    print(list_cur)
                    sheet.append(list_cur)
book.save(filename=sheet.title+'.xlsx')

['2019-03-23', '잠실', '14:00', '두산', '5', 1, '한화', '4', 0, '허경민&1990-08-26', '정수빈&1990-10-07', '박건우&1990-09-08', '김재환&1988-09-22', '오재일&1986-10-29', '페르난데스&1988-04-27', '오재원&1985-02-09', '박세혁&1990-01-09', '김재호&1985-03-21', '정근우&1982-10-02', '송광민&1983-06-24', '호잉&1989-05-18', '김태균&1982-05-29', '이성열&1984-07-13', '하주석&1994-02-25', '김민하&1989-02-25', '최재훈&1989-08-27', '정은원&2000-01-17', '3.3', '1.7', '8.6', '46.9']
['2019-03-23', '문학', '14:00', 'SK', '7', 1, 'KT', '4', 0, '노수광&1990-08-06', '한동민&1989-08-09', '최정&1987-02-28', '로맥&1985-09-30', '이재원&1988-02-24', '정의윤&1986-07-25', '김강민&1982-09-13', '최항&1994-01-03', '김성현&1987-03-09', '황재균&1987-07-28', '박경수&1984-03-31', '강백호&1999-07-29', '로하스&1990-05-24', '유한준&1981-07-01', '윤석민&1985-09-04', '오승택&1991-11-18', '장성우&1990-01-17', '배정대&1995-06-12', '3.8', '2.6', '16.9', '54.8']
['2019-03-23', '사직', '14:00', '키움', '7', 0, '롯데', '4', 1, '서건창&1989-08-22', '김하성&1995-10-17', '박병호&1986-07-10', '샌즈&1987-09-28', '장영석&1990-05-14', '임병욱&1995-09-30', '허정협&1990-02-1

['2019-07-27', '잠실', '18:00', '두산', '12', 1, 'KIA', '1', 0, '박건우&1990-09-08', '페르난데스&1988-04-27', '오재일&1986-10-29', '김재환&1988-09-22', '최주환&1988-02-28', '허경민&1990-08-26', '박세혁&1990-01-09', '김재호&1985-03-21', '정수빈&1990-10-07', '김주찬&1981-03-25', '김선빈&1989-12-18', '터커&1990-07-06', '최형우&1983-12-16', '안치홍&1990-07-02', '나지완&1985-05-19', '이창진&1991-03-04', '한승택&1994-06-21', '박찬호&1995-06-05', '28.4', '0.6', '4.7', '76.6']
['2019-07-27', '사직', '18:00', 'SK', '4', 0, '롯데', '0', 1, '김강민&1982-09-13', '고종욱&1989-01-11', '최정&1987-02-28', '로맥&1985-09-30', '이재원&1988-02-24', '정의윤&1986-07-25', '나주환&1984-06-14', '김성현&1987-03-09', '노수광&1990-08-06', '민병헌&1987-03-10', '손아섭&1988-03-18', '전준우&1986-02-25', '이대호&1982-06-21', '윌슨&1990-07-29', '김문호&1987-06-22', '신본기&1989-03-21', '안중열&1995-09-01', '강로한&1992-05-13', '25.9', '7.3', '16.6', '98.5']
['2019-07-27', '대구', '18:00', '삼성', '7', 1, '한화', '6', 0, '김상수&1990-03-23', '박해민&1990-02-24', '김헌곤&1988-11-09', '러프&1986-07-28', '이원석&1986-10-21', '윌리엄슨&1990-07-15', '김동엽&1990

['2019-08-28', '울산', '18:30', '롯데', '4', 1, 'LG', '3', 0, '고승민&2000-08-11', '손아섭&1988-03-18', '민병헌&1987-03-10', '이대호&1982-06-21', '전준우&1986-02-25', '한동희&1999-06-01', '신본기&1989-03-21', '안중열&1995-09-01', '강로한&1992-05-13', '이천웅&1988-10-20', '정주현&1990-10-13', '이형종&1989-06-07', '김현수&1988-01-12', '채은성&1990-02-06', '페게로&1987-02-22', '김민성&1988-12-17', '유강남&1992-07-15', '오지환&1990-03-12', '23.2', '0.4', '4.3', '94.9']
['2019-08-28', '청주', '18:30', '한화', '2', 1, '키움', '1', 0, '장진혁&1993-09-30', '정근우&1982-10-02', '송광민&1983-06-24', '호잉&1989-05-18', '김태균&1982-05-29', '백창수&1988-05-09', '최재훈&1989-08-27', '오선진&1989-07-07', '정은원&2000-01-17', '이정후&1998-08-20', '서건창&1989-08-22', '김하성&1995-10-17', '박병호&1986-07-10', '샌즈&1987-09-28', '김규민&1993-01-26', '송성문&1996-08-29', '이지영&1986-02-27', '김혜성&1999-01-27', '24.8', '0.6', '4.7', '77.9']
['2019-08-29', '대구', '18:30', '삼성', '5', 1, 'SK', '4', 0, '박계범&1996-01-11', '박해민&1990-02-24', '구자욱&1993-02-12', '러프&1986-07-28', '김헌곤&1988-11-09', '윌리엄슨&1990-07-15', '이원석&1986-10

In [75]:
print(dict_match)

{'2019': {'3월': {'23일': {'잠실': {'두산 한화': {'14:00': ['두산', '5', 1, '한화', '4', 0, '허경민&1990-08-26', '정수빈&1990-10-07', '박건우&1990-09-08', '김재환&1988-09-22', '오재일&1986-10-29', '페르난데스&1988-04-27', '오재원&1985-02-09', '박세혁&1990-01-09', '김재호&1985-03-21', '정근우&1982-10-02', '송광민&1983-06-24', '호잉&1989-05-18', '김태균&1982-05-29', '이성열&1984-07-13', '하주석&1994-02-25', '김민하&1989-02-25', '최재훈&1989-08-27', '정은원&2000-01-17', '3.3', '1.7', '8.6', '46.9']}}, '문학': {'SK KT': {'14:00': ['SK', '7', 1, 'KT', '4', 0, '노수광&1990-08-06', '한동민&1989-08-09', '최정&1987-02-28', '로맥&1985-09-30', '이재원&1988-02-24', '정의윤&1986-07-25', '김강민&1982-09-13', '최항&1994-01-03', '김성현&1987-03-09', '황재균&1987-07-28', '박경수&1984-03-31', '강백호&1999-07-29', '로하스&1990-05-24', '유한준&1981-07-01', '윤석민&1985-09-04', '오승택&1991-11-18', '장성우&1990-01-17', '배정대&1995-06-12', '3.8', '2.6', '16.9', '54.8']}}, '사직': {'키움 롯데 ': {'14:00': ['키움', '7', 0, '롯데', '4', 1, '서건창&1989-08-22', '김하성&1995-10-17', '박병호&1986-07-10', '샌즈&1987-09-28', '장영석&1990-05-14', '임병욱&1995

In [76]:
# year='2018'
# team='KIA'
# ymd='2018-08-08'
# time='18:30'
# url = 'http://www.statiz.co.kr/team.php?opt=0&sopt=4&year='+year+'&team='+team+'&mdate='+ymd
# driver = webdriver.Chrome('./chromedriver')
# driver.get(url)
# soup = BeautifulSoup(driver.page_source,'html.parser')
# data_box = soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div')
# box = data_box[0]
# if data_box[0].select_one('h3 > font').text != time:
#     box = data_box[1]
# print(box)
# atags = box.select('table > tbody > tr > td > a')
# print(atags)


In [77]:
# year='2018'
# month = '8월'
# day = '8일'
# time = '18:30'
# match = 'KIA LG'
# list_temp = getBatter(year, month[:-1], day[:-1], time, match.split(' ')[0])
# print(list_temp)